In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time

chrome_path = "C:\\Users\\iverson\\Documents\\Python Scripts\\fundrich-parser\\selenium_driver_chrome\\chromedriver.exe" 
web = webdriver.Chrome(chrome_path)

web.get('https://www.fundrich.com.tw/FundWeb/Overview/Investment.aspx')
web.set_window_position(0,0) #瀏覽器位置
web.set_window_size(700,700) #瀏覽器大小
time.sleep(5)

username = web.find_element_by_xpath("//input[@type='text']")
password = web.find_element_by_xpath("//input[@type='password']")


username.send_keys("username")
password.send_keys("password")
web.find_element_by_xpath('//*[@id="content"]/div/div/div[2]/div/div[2]/div/div/button').click()
time.sleep(5)
web.get("https://www.fundrich.com.tw/FundWeb/Overview/Investment.aspx")

#web.close()

In [2]:
web.find_element_by_xpath('//*[@id="table1_length"]/label/select/option[3]').click()
time.sleep(2)

spn_INCOME_RATE = web.find_element_by_xpath('//*[@id="spn_INCOME_RATE"]').text
print(spn_INCOME_RATE)


-2.47%


In [3]:
content_data = []
header_data = []
table = web.find_element_by_xpath('//*[@id="table1"]')
table_header = [header.text for header in table.find_elements_by_tag_name('th')]
table_content = [header.text for header in table.find_elements_by_tag_name('td')]


In [4]:
table_content

['NN (L) 新興市場債券基金X股美元(月配息)',
 '91.78(USD)\n2018/06/08',
 '2,819.2839\n29.7825',
 '3.547\n3.547',
 '新臺幣',
 '10,000\n10,000',
 '9,695\n72',
 '-305',
 '-2.33%\n-3.05%',
 '買回\n轉換\n轉申購',
 '安本環球印度債券基金X 月配息 美元',
 '9.1534(USD)\n2018/06/08',
 '279.392\n29.7825',
 '35.792\n35.792',
 '新臺幣',
 '10,000\n10,000',
 '9,757\n50',
 '-243',
 '-1.93%\n-2.43%',
 '買回\n轉換\n轉申購',
 '安聯收益成長基金-AM穩定月收類股(美元)',
 '9.25(USD)\n2018/06/08',
 '267.8853\n29.7825',
 '25.328\n25.328',
 '新臺幣',
 '6,785\n6,785',
 '6,977\n73',
 '192',
 '3.91%\n2.83%',
 '買回\n轉換\n轉申購',
 '富蘭克林坦伯頓全球債券基金美元A(Mdis)股',
 '17.4(USD)\n2018/06/08',
 '531.0674\n29.7825',
 '18.83\n18.83',
 '新臺幣',
 '10,000\n10,000',
 '9,758\n0',
 '-242',
 '-2.42%\n-2.42%',
 '買回\n轉換\n轉申購',
 '富蘭克林坦伯頓全球投資系列－全球債券總報酬基金美元A(Mdis)股',
 '14.33(USD)\n2018/06/08',
 '440.4704\n29.7825',
 '22.703\n22.703',
 '新臺幣',
 '10,000\n10,000',
 '9,689\n0',
 '-311',
 '-3.11%\n-3.11%',
 '買回\n轉換\n轉申購',
 '富蘭克林坦伯頓全球投資系列－新興國家固定收益基金美元A (Mdis)股',
 '8.21(USD)\n2018/06/08',
 '260.6339\n29.7825',
 '38.368\n38.3

In [5]:
table_header

['基金名稱',
 '參考淨值\n淨值日期',
 '平均\n成本\n參考\n匯率',
 '庫存單位\n可買回單位',
 '交易\n幣別',
 '投資本金\n台幣本金',
 '參考市值\n累計配息金額',
 '參考損益',
 '報酬率\n(含息)\n報酬率\n(不含息)',
 '功能']

In [6]:
new_table_header=[]
for txt in table_header:
    if txt == '平均\n成本\n參考\n匯率':
        txt = '平均成本\n參考匯率'
    if txt == '交易\n幣別':
        txt = '交易幣別'
    if txt == '報酬率\n(含息)\n報酬率\n(不含息)':
        txt = '報酬率(含息)\n報酬率(不含息)'
    new_table_header.extend(txt.split("\n")) 

print(new_table_header)

['基金名稱', '參考淨值', '淨值日期', '平均成本', '參考匯率', '庫存單位', '可買回單位', '交易幣別', '投資本金', '台幣本金', '參考市值', '累計配息金額', '參考損益', '報酬率(含息)', '報酬率(不含息)', '功能']


In [7]:
new_table_content=[]
for txt in table_content:
    if txt == '買回\n轉換\n轉申購':
        txt =  '買回轉換轉申購'
    new_table_content.extend(txt.split("\n")) 
    
    
new_table_content = np.array(new_table_content).reshape(-1,len(new_table_header))
    
print(new_table_content)

[['NN (L) 新興市場債券基金X股美元(月配息)' '91.78(USD)' '2018/06/08' '2,819.2839'
  '29.7825' '3.547' '3.547' '新臺幣' '10,000' '10,000' '9,695' '72' '-305'
  '-2.33%' '-3.05%' '買回轉換轉申購']
 ['安本環球印度債券基金X 月配息 美元' '9.1534(USD)' '2018/06/08' '279.392' '29.7825'
  '35.792' '35.792' '新臺幣' '10,000' '10,000' '9,757' '50' '-243' '-1.93%'
  '-2.43%' '買回轉換轉申購']
 ['安聯收益成長基金-AM穩定月收類股(美元)' '9.25(USD)' '2018/06/08' '267.8853' '29.7825'
  '25.328' '25.328' '新臺幣' '6,785' '6,785' '6,977' '73' '192' '3.91%'
  '2.83%' '買回轉換轉申購']
 ['富蘭克林坦伯頓全球債券基金美元A(Mdis)股' '17.4(USD)' '2018/06/08' '531.0674' '29.7825'
  '18.83' '18.83' '新臺幣' '10,000' '10,000' '9,758' '0' '-242' '-2.42%'
  '-2.42%' '買回轉換轉申購']
 ['富蘭克林坦伯頓全球投資系列－全球債券總報酬基金美元A(Mdis)股' '14.33(USD)' '2018/06/08'
  '440.4704' '29.7825' '22.703' '22.703' '新臺幣' '10,000' '10,000' '9,689'
  '0' '-311' '-3.11%' '-3.11%' '買回轉換轉申購']
 ['富蘭克林坦伯頓全球投資系列－新興國家固定收益基金美元A (Mdis)股' '8.21(USD)' '2018/06/08'
  '260.6339' '29.7825' '38.368' '38.368' '新臺幣' '10,000' '10,000' '9,381'
  '74' '-619' '-5.4

In [8]:
new_df = pd.DataFrame(columns=new_table_header, data=new_table_content)

In [9]:
new_df

,基金名稱,參考淨值,淨值日期,平均成本,參考匯率,庫存單位,可買回單位,交易幣別,投資本金,台幣本金,參考市值,累計配息金額,參考損益,報酬率(含息),報酬率(不含息),功能
0,NN (L) 新興市場債券基金X股美元(月配息),91.78(USD),2018/06/08,"2,819.2839",29.7825,3.547,3.547,新臺幣,"10,000","10,000","9,695",72,-305,-2.33%,-3.05%,買回轉換轉申購
1,安本環球印度債券基金X 月配息 美元,9.1534(USD),2018/06/08,279.392,29.7825,35.792,35.792,新臺幣,"10,000","10,000","9,757",50,-243,-1.93%,-2.43%,買回轉換轉申購
2,安聯收益成長基金-AM穩定月收類股(美元),9.25(USD),2018/06/08,267.8853,29.7825,25.328,25.328,新臺幣,"6,785","6,785","6,977",73,192,3.91%,2.83%,買回轉換轉申購
3,富蘭克林坦伯頓全球債券基金美元A(Mdis)股,17.4(USD),2018/06/08,531.0674,29.7825,18.83,18.83,新臺幣,"10,000","10,000","9,758",0,-242,-2.42%,-2.42%,買回轉換轉申購
4,富蘭克林坦伯頓全球投資系列－全球債券總報酬基金美元A(Mdis)股,14.33(USD),2018/06/08,440.4704,29.7825,22.703,22.703,新臺幣,"10,000","10,000","9,689",0,-311,-3.11%,-3.11%,買回轉換轉申購
5,富蘭克林坦伯頓全球投資系列－新興國家固定收益基金美元A (Mdis)股,8.21(USD),2018/06/08,260.6339,29.7825,38.368,38.368,新臺幣,"10,000","10,000","9,381",74,-619,-5.45%,-6.19%,買回轉換轉申購
6,富蘭克林坦伯頓新興國家固定收益基金美元A (Qdis)股,14.3(USD),2018/06/08,439.2708,29.7825,22.765,22.765,新臺幣,"10,000","10,000","9,695",0,-305,-3.05%,-3.05%,買回轉換轉申購
7,富達基金－亞洲高收益基金 (A股月配息),8.021(USD),2018/06/08,244.2599,29.7825,40.94,40.94,新臺幣,"10,000","10,000","9,780",52,-220,-1.68%,-2.2%,買回轉換轉申購
8,復華新興市場高收益債券基金-人民幣配息,12.66(CNY),2018/06/07,13.2774,4.635,143.1,143.1,人民幣,"1,900","8,807","1,811.65",13.95,-88.35,-3.92%,-4.65%,買回轉換轉申購
9,施羅德環球基金系列 -歐洲股息基金 A 類股份 - 配息單位(歐元),52.8351(EUR),2018/06/08,"1,904.7619",35.02,5.25,5.25,新臺幣,"10,000","10,000","9,714",0,-286,-2.86%,-2.86%,買回轉換轉申購
